In [1]:
from sklearn import metrics
import pandas as pd
from torch import nn
from pyarabic.araby import tokenize
import numpy as np
import pickle
import spacy
import torch

# from model_building import Classifier 
from pre_processing_post import processPost
from feature_extraction import get_ngram_features, get_word_embedding_features, avg_word_vector
from gru_model import ArabicDataset, Classifier, evaluate, train

In [2]:
# needed functions
def print_report(y_pred, y_test):
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [3]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

c:\Users\Nada\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'ar_pipeline' (0.0.0) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
train_data = pd.read_csv('./DataSet/train.csv',sep=',',header=0)
test_data = pd.read_csv('./DataSet/dev.csv',sep=',',header=0)
train_data['text'] = train_data['text'].apply(lambda x: processPost(x))
test_data['text'] = test_data['text'].apply(lambda x: processPost(x))

creat vocablary

In [5]:
train_data_tokenized = train_data['text'].apply(tokenize)
test_data_tokenized = test_data['text'].apply(tokenize)
#merge all the sentences in one list
vocab = [item for sublist in train_data_tokenized for item in sublist]
vocab = list(set(vocab))
vocab.append('<فراغ>')
vocab.insert(0, '<مجهول>')
word2index = {word: i for i, word in enumerate(vocab)}

In [6]:
weights_train_matrix = []
for word in vocab:
  weights_train_matrix.append(nlp(word).vector)

weights_train_matrix = torch.from_numpy(np.array(weights_train_matrix))
weights_train_matrix.size()

torch.Size([12538, 100])

td-idf feature

In [108]:
# load naive bayes model
with open('./models/NaiveBayes_tfidf.sav', 'rb') as f:
    naive_bayes_model = pickle.load(f)

with open('./models/TFIDFVectorizer.sav', 'rb') as f:
    word_vectorizer = pickle.load(f)

X_test_tfidf = word_vectorizer.transform(test_data['text'])
y_pred = naive_bayes_model.predict_proba(X_test_tfidf)

Ara2Vec embedding

In [109]:
test_data_embeddings = np.array([np.array([nlp(i).vector for i in ls]) for ls in test_data["text"]], dtype=object)
_, X_test_vect_avg = avg_word_vector([], test_data_embeddings)

# load SVC model
with open('./models/RandomForest_Ara2Vec.sav', 'rb') as f:
    svc_model = pickle.load(f)

y_pred += svc_model.predict_proba(X_test_vect_avg)

In [23]:
#load GRU model
with open('./models/GRU_Ara2Vec.pth', 'rb') as f: 
    # device = torch.device("cpu")
    gru_model = Classifier(weights_train_matrix)
    gru_model.load_state_dict(torch.load(f)) 
    # gru_model.eval()


test_data_tokenized_as_num = test_data_tokenized.apply(lambda x: [word2index[word] for word in x if word in word2index])
print(test_data_tokenized_as_num[0])
test_dataset = ArabicDataset(list(test_data_tokenized_as_num), test_data['stance'] + 1, word2index['<فراغ>'])
print(test_dataset[0])
y_pred_ = evaluate(gru_model,test_dataset)
print(y_pred_)

[7607, 267, 4431, 4927, 9127, 12436, 1611, 1750, 6395, 8674, 4496, 7963, 5897, 6321, 1475, 2432, 7607, 267, 10169, 9314, 421, 240, 5694, 6321, 6020, 4927, 9127, 4431, 11147, 1611, 1750, 8411, 300, 10965, 7181, 1611, 7963, 6448]
(tensor([ 7607,   267,  4431,  4927,  9127, 12436,  1611,  1750,  6395,  8674,
         4496,  7963,  5897,  6321,  1475,  2432,  7607,   267, 10169,  9314,
          421,   240,  5694,  6321,  6020,  4927,  9127,  4431, 11147,  1611,
         1750,  8411,   300, 10965,  7181,  1611,  7963,  6448, 12537, 12537,
        12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537, 12537,
        12537, 12537, 12537, 12537], dtype=torch.int32), tensor(2))


100%|██████████| 32/32 [00:01<00:00, 24.43it/s]


accuracy: 0.219
              precision    recall  f1-score   support

           0       0.07      0.30      0.12        70
           1       0.12      0.56      0.20       126
           2       0.82      0.16      0.27       804

    accuracy                           0.22      1000
   macro avg       0.34      0.34      0.20      1000
weighted avg       0.68      0.22      0.25      1000


Test Accuracy: 0.21899999678134918
[[-1.922905    0.9292855   1.3081696 ]
 [-2.780834    1.2828125   1.4526263 ]
 [-0.27978054  0.13587283 -0.15097593]
 ...
 [-2.2708514   2.081671   -0.48762155]
 [-0.26863557  0.39028868 -0.23718561]
 [ 0.08276586  0.16059619 -0.13214381]]


In [112]:
y_pred /= 2
y_pred = np.argmax(y_pred, axis=1) - 1
print_report(y_pred, test_data['stance'])

              precision    recall  f1-score   support

          -1       0.43      0.04      0.08        70
           0       0.38      0.13      0.20       126
           1       0.82      0.97      0.89       804

    accuracy                           0.80      1000
   macro avg       0.54      0.38      0.39      1000
weighted avg       0.74      0.80      0.75      1000

accuracy: 0.799
